In [1]:
#pip install google-cloud-bigquery これ必要
#pip install pydata_google_auth
#pip install db-dtypes

In [2]:
#インポート文
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from lightgbm import LGBMRanker

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

from sklearn.metrics import confusion_matrix

import optuna

import pydata_google_auth
from google.cloud import bigquery 

import pickle

c:\Users\kawam\miniforge3\envs\keiba-d\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)

In [4]:
client = bigquery.Client(project='keiba-381006', credentials=credentials)

In [5]:
horse_query = '''
    SELECT * 
    FROM horse_dataset.main_horse_table 
    LIMIT 1000
    '''
horse_df = client.query(horse_query).to_dataframe()

In [6]:
race_query = '''
    SELECT * 
    FROM race_dataset.main_race_table 
    LIMIT 1000
    '''
race_df = client.query(race_query).to_dataframe()

In [7]:
#データのマージ
race_df_for_merge = race_df[['race_id',
                             "distance",
                            ]]

merged_horse_df = pd.merge(horse_df, race_df_for_merge, on='race_id')

In [8]:
#使う特徴量の設定
id = {"race_id",
      "horse_id"
    }

feature = [
           "race_id",
           "horse_id",
           "frame_number",
           "horse_number",
           "horse_weight",
           "distance",
            ]

target = ['rank']

X_id = merged_horse_df[feature]
X = X_id.drop(id, axis =1)
y = merged_horse_df[target]

In [11]:
X = X.astype(float)
y = y.astype(float)
model = lgb.LGBMRegressor() 
model.fit(X, y) 

LGBMRegressor()

In [12]:
X_id= X_id.astype(float)
y= y.astype(float)

In [13]:
result = model.predict(X)
result[0]

7.738006204156375

In [20]:
frame_number= 1
horse_number = 2
horse_weight = 410
distance = 1600

result = model.predict([[frame_number,horse_number,horse_weight,distance]])

result #array([9.59372813])
result[0] #9.59372812548868

9.59372812548868

In [14]:
with open('model.pickle', 'wb') as f:
    pickle.dump(model, f)